<a href="https://colab.research.google.com/github/neostrange/CveParse/blob/main/CVE_to_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neo4j    

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.5/187.5 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.8.0-py3-none-any.whl size=258407 sha256=a038bed7f407634c6d546d4495c7207a4fddcac25d26a885f232362eb06f3bd2
  Stored in directory: /root/.cache/pip/wheels/6b/b5/da/73f634944e04e625954d101cb175ac1aeb9b29751a37d3383e
Successfully built neo4j


In [2]:
from neo4j import GraphDatabase 

In [3]:
uri = "neo4j+s://2d6be118.databases.neo4j.io"
pwd = "B1q_0RIcKGQInCRLe6Qpk3Is3F52n4jrzLfhS4HlsUs"
driver = GraphDatabase.driver(uri, auth=("neo4j", pwd))    

In [4]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [5]:
uri = "neo4j+s://2d6be118.databases.neo4j.io"
pwd = "B1q_0RIcKGQInCRLe6Qpk3Is3F52n4jrzLfhS4HlsUs"
user = 'neo4j'
conn = Neo4jConnection(uri, user, pwd)

## **CONSTRAINTS**

In [6]:
explore_reviews_csv_query = """
CREATE CONSTRAINT CVE IF NOT EXISTS FOR (cve:CVE) REQUIRE cve.id IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [7]:
explore_reviews_csv_query = """
CREATE CONSTRAINT references IF NOT EXISTS FOR (ref:Reference) REQUIRE ref.url IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [8]:
explore_reviews_csv_query = """
CREATE CONSTRAINT descriptions IF NOT EXISTS FOR (desc:Description) REQUIRE desc.id IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [9]:
explore_reviews_csv_query = """
CREATE CONSTRAINT tags IF NOT EXISTS FOR (t:Tag) REQUIRE t.tag IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [10]:
explore_reviews_csv_query = """
CREATE CONSTRAINT CONFIG IF NOT EXISTS FOR (config:CONFIG) REQUIRE config.id IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [11]:
explore_reviews_csv_query = """
CREATE CONSTRAINT CPE IF NOT EXISTS FOR (cpe:CPE) REQUIRE cpe.cpe23Uri IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [12]:
explore_reviews_csv_query = """
CREATE CONSTRAINT CWE IF NOT EXISTS FOR (cwe:CWE) REQUIRE cwe.CWE IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [13]:
explore_reviews_csv_query = """
CREATE CONSTRAINT CPE IF NOT EXISTS FOR (cpe:CPE) REQUIRE cpe.CPE IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


# **CVE Data**

script_1.cypher

In [14]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/nvd3.json", null, {failOnError:false})
YIELD value
WITH value  AS data

UNWIND data.CVE_Items as item
MERGE (cve:CVE {id: item.cve.CVE_data_meta.ID})
ON CREATE SET cve.assigner = item.cve.CVE_data_meta.ASSIGNER

WITH cve, item
UNWIND item.cve.problemtype.problemtype_data AS problem
UNWIND problem.description AS problem_desc
MERGE (cve)-[:PROBLEM_TYPE]->(problemType:ProblemType {lang: problem_desc.lang, value: problem_desc.value})
//MERGE (cve)-[:PROBLEM_TYPE]->(problemType)


WITH item, cve
UNWIND item.cve.references.reference_data AS ref
MERGE (reference:Reference {url: ref.url})
ON CREATE SET reference.name = ref.name, reference.refsource = ref.refsource

WITH item, cve, reference, ref
UNWIND ref.tags AS tag
MERGE (tagNode:Tag {name: tag, id: tag})
MERGE (reference)-[:TAGGED_WITH]->(tagNode)

WITH item, cve, reference
UNWIND item.cve.description.description_data AS desc
MERGE (description:Description {lang: desc.lang, value:desc.value, cveId:cve.id})
//ON CREATE SET description.value = desc.value

//WITH item, cve, reference, description
MERGE (cve)-[:HAS_REFERENCE]->(reference)
MERGE (cve)-[:HAS_DESCRIPTION]->(description)



WITH item, cve,item.impact.baseMetricV3 as baseMetricV3, item.impact.baseMetricV3.cvssV3 as cvssV3
MERGE (cve)-[:HAS_IMPACT]->(impact:IMPACT {exploitabilityScore: baseMetricV3.exploitabilityScore, 
                                           impactScore: baseMetricV3.impactScore, cveId: cve.id
})-[:HAS_CVSSV3]->(cvssv3:CVSSV3 {version: cvssV3.version,
                                              vectorString:cvssV3.vectorString,
                                              attackVector:cvssV3.attackVector,
                                              attackComplexity:cvssV3.attackComplexity,
                                              privilegesRequired:cvssV3.privilegesRequired,
                                              userInteraction:cvssV3.userInteraction,
                                              scope:cvssV3.scope,
                                              confidentialityImpact:cvssV3.confidentialityImpact,
                                              integrityImpact:cvssV3.integrityImpact,
                                              availabilityImpact:cvssV3.availabilityImpact,
                                              baseScore:cvssV3.baseScore,
                                              baseSeverity:cvssV3.baseSeverity 
})
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


script_2.cypher

In [15]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/nvd3.json", null, {failOnError:false})
YIELD value
WITH value  AS data

UNWIND data.CVE_Items as item
MERGE (cve:CVE {id: item.cve.CVE_data_meta.ID})
ON CREATE SET cve.assigner = item.cve.CVE_data_meta.ASSIGNER

WITH item.configurations AS configurations, cve
MERGE (config:Config {version: configurations.CVE_data_version, cveId: cve.id})
MERGE (cve)-[:HAS_CONFIG]->(config)

WITH configurations, config, cve
UNWIND configurations.nodes as node
  // Each node has operator, cpe_match and  children
  //create operator for the node
  CREATE (op:Operator {operator: node.operator})
  CREATE (config)-[:HAS_OPERATOR]->(op)
  //create cpe_match for the node
  FOREACH (cpe_match in node.cpe_match |
    CREATE (cpeMatch:CpeMatch {vulnerable: cpe_match.vulnerable, uri: cpe_match.cpe23Uri})
    CREATE (cpeMatch)-[:HAS_OPERATOR]->(op))

    //children in each node -  each children then has operator, cpe_match and children
    FOREACH (child IN node.children | 
      CREATE (op1:Operator {operator: child.operator})
      CREATE (op)-[:HAS_OPERATOR]->(op1)
      FOREACH (cpe in child.cpe_match |
        CREATE (cpeMatch:CpeMatch {vulnerable: cpe.vulnerable, uri: cpe.cpe23Uri})
        MERGE (op1)-[:CPE_MATCH]->(cpeMatch))
)
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


# **CWE Data**

Script_CWE.cypher

In [16]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/CWE.json", null, {failOnError:false})
YIELD value
WITH value  AS data
//return data.CWE_Items[0].CWE.cwe_id
UNWIND data.CWE_Items as item
MERGE (cwe:CWE {cwe_id: item.CWE.cwe_id})
SET 
    cwe.name = item.CWE.name,
    cwe.weakness_abstraction = item.CWE.weakness_abstraction, 
    cwe.status = item.CWE.status, 
    cwe.description = item.CWE.description,  
    cwe.extended_description = item.CWE.extended_description

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


# **Mapping btween CVE and CWE**

In [17]:
explore_reviews_csv_query = """
MERGE (problemType:ProblemType {value:"CWE-79"})
MERGE (cwe:CWE {cwe_id:"79"})
MERGE (problemType)-[:MAPPED_WITH]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [18]:
explore_reviews_csv_query = """
MERGE (problemType:ProblemType {value:"CWE-89"})
MERGE (cwe:CWE {cwe_id:"89"})
MERGE (problemType)-[:MAPPED_WITH]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [19]:
explore_reviews_csv_query = """
MERGE (problemType:ProblemType {value:"CWE-94"})
MERGE (cwe:CWE {cwe_id:"94"})
MERGE (problemType)-[:MAPPED_WITH]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [20]:
explore_reviews_csv_query = """
MERGE (problemType:ProblemType {value:"CWE-200"})
MERGE (cwe:CWE {cwe_id:"200"})
MERGE (problemType)-[:MAPPED_WITH]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [21]:
explore_reviews_csv_query = """
MERGE (problemType:ProblemType {value:"CWE-284"})
MERGE (cwe:CWE {cwe_id:"284"})
MERGE (problemType)-[:MAPPED_WITH]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [22]:
explore_reviews_csv_query = """
MERGE (problemType:ProblemType {value:"CWE-294"})
MERGE (cwe:CWE {cwe_id:"294"})
MERGE (problemType)-[:MAPPED_WITH]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [23]:
explore_reviews_csv_query = """
MERGE (cwe1:CWE { cwe_id:"79"})
MERGE (cwe2:CWE { cwe_id:"89"})
MERGE (cwe1)-[:HAS_RELATED_WEAKNESS{ NATURE:"ChildOf",CWEـID:"74",VIEWـID:"1003",ORDINAL:"Primary"}]->(cwe2)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [24]:
explore_reviews_csv_query = """
MERGE (cwe1:CWE { cwe_id:"79"})
MERGE (cwe2:CWE { cwe_id:"94"})
MERGE (cwe1)-[:HAS_RELATED_WEAKNESS{ NATURE:"ChildOf",CWEـID:"74",VIEWـID:"1000",ORDINAL:"Primary"}]->(cwe2)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [25]:
explore_reviews_csv_query = """
MERGE (cwe1:CWE { cwe_id:"79"})
MERGE (cwe2:CWE { cwe_id:"94"})
MERGE (cwe1)-[:HAS_RELATED_WEAKNESS{ NATURE:"ChildOf",CWEـID:"74",VIEWـID:"1003",ORDINAL:"Primary"}]->(cwe2)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [26]:
explore_reviews_csv_query = """
MERGE (cwe1:CWE { cwe_id:"89"})
MERGE (cwe2:CWE { cwe_id:"94"})
MERGE (cwe1)-[:HAS_RELATED_WEAKNESS{ NATURE:"ChildOf",CWEـID:"74",VIEWـID:"1003",ORDINAL:"Primary"}]->(cwe2)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


Extract the entities from unstructured data (extended_description) in CWE to structured data by using entity-fishing



In [27]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/cwedata.json", null, {failOnError:false})
YIELD value
WITH value  AS data
//return data.Extended_Descriptions[0].CWE.runtime
UNWIND data.Extended_Descriptions as extended_descript
MERGE (cwedata:CWE {runtime: extended_descript.CWE.runtime})
SET 
    cwedata.software = extended_descript.CWE.software,
    cwedata.version = extended_descript.CWE.version,
    cwedata.date = extended_descript.CWE.date,
    cwedata.nbest = extended_descript.CWE.nbest,
    cwedata.text = extended_descript.CWE.text

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [28]:
explore_reviews_csv_query = """
MERGE (cwe:CWE {cwe_id:"79"})
MERGE (cwedata:CWE {runtime:2992})
MERGE (cwe)-[:MAPPED_WITH]->(cwedata)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.04878159541353045,source:"wikipedia-en",category:"Hacking (computer security)",page_id:11647669}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.026132997553802943,source:"wikipedia-en",category:"Web security exploits",page_id:5131853}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.001742199837897568,source:"wikipedia-en",category:"Unicode",page_id:720904}]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [29]:
MERGE (cwedata)-[:HAS_ENTITY{rawName:"",offsetStart:,offsetEnd:,confidence_score:,wikipediaExternalRef:,wikidataId:""}]->(cwe)

SyntaxError: ignored

In [30]:
explore_reviews_csv_query = """

MERGE (cwedata)-[:HAS_ENTITY{rawName:"Cross-site scripting",offsetStart:0,offsetEnd:20,confidence_score:0.719,wikipediaExternalRef:241154,wikidataId:"Q371199"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"XSS",offsetStart:22,offsetEnd:25,confidence_score:0.8255,wikipediaExternalRef:241154,wikidataId:"Q371199"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"vulnerabilities",offsetStart:27,offsetEnd:42,confidence_score:0.566,wikipediaExternalRef:1129827,wikidataId:"Q631425"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"Untrusted",offsetStart:58,offsetEnd:67,confidence_score:0.5761,wikipediaExternalRef:34973773,wikidataId:"Q4976859"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"enters",offsetStart:73,offsetEnd:79,confidence_score:0.3761,wikipediaExternalRef:25162935,wikidataId:"Q4745451"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"web application",offsetStart:82,offsetEnd:97,confidence_score:0.8604,wikipediaExternalRef:288311,wikidataId:"Q189210"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"web request",offsetStart:116,offsetEnd:127,confidence_score:0.6576,wikipediaExternalRef:13443,wikidataId:"Q8777"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"2",type:"MEASURE",offsetStart:129,offsetEnd:130,confidence_score:0}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"web application",offsetStart:136,offsetEnd:151,confidence_score:0.8604,wikipediaExternalRef:288311,wikidataId:"Q189210"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"dynamically generates",offsetStart:152,offsetEnd:173,confidence_score:0.4623,wikipediaExternalRef:2015575,wikidataId:"Q4291515"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"web page",offsetStart:176,offsetEnd:184,confidence_score:0.7056,wikipediaExternalRef:21076839,wikidataId:"Q36774"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"untrusted",offsetStart:204,offsetEnd:213,confidence_score:0.5761,wikipediaExternalRef:34973773,wikidataId:"Q4976859"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"executable",offsetStart:321,offsetEnd:331,confidence_score:0.5397,wikipediaExternalRef:217392,wikidataId:"Q778586"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"web browser",offsetStart:337,offsetEnd:348,confidence_score:0.8945,wikipediaExternalRef:33173,wikidataId:"Q6368"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"JavaScript",offsetStart:358,offsetEnd:368,confidence_score:0.915,wikipediaExternalRef:9845,wikidataId:"Q2005"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"HTML",offsetStart:370,offsetEnd:374,confidence_score:0.4357,wikipediaExternalRef:13191,wikidataId:"Q8811"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"tags",offsetStart:375,offsetEnd:379,confidence_score:0.5171,wikipediaExternalRef:274393,wikidataId:"Q179551"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"HTML attributes",offsetStart:381,offsetEnd:396,confidence_score:0.8042,wikipediaExternalRef:26808274,wikidataId:"Q5636097"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"Flash",offsetStart:412,offsetEnd:417,confidence_score:0.6094,wikipediaExternalRef:20947,wikidataId:"Q165658"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"ActiveX",offsetStart:419,offsetEnd:426,confidence_score:0.8932,wikipediaExternalRef:93497,wikidataId:"Q302022"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"web page",offsetStart:466,offsetEnd:474,confidence_score:0.7056,wikipediaExternalRef:21076839,wikidataId:"Q36774"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"web browser",offsetStart:485,offsetEnd:496,confidence_score:0.8945,wikipediaExternalRef:33173,wikidataId:"Q6368"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"malicious",offsetStart:513,offsetEnd:522,confidence_score:0.5011,wikipediaExternalRef:20901,wikidataId:"Q14001"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"injected",offsetStart:539,offsetEnd:547,confidence_score:0.5029,wikipediaExternalRef:1065470,wikidataId:"Q2920753"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"untrusted",offsetStart:558,offsetEnd:567,confidence_score:0.5761,wikipediaExternalRef:34973773,wikidataId:"Q4976859"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"web page",offsetStart:607,offsetEnd:615,confidence_score:0.7056,wikipediaExternalRef:21076839,wikidataId:"Q36774"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"web serve",offsetStart:637,offsetEnd:647,confidence_score:0.8665,wikipediaExternalRef:33455,wikidataId:"Q11288"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"web browser",offsetStart:662,offsetEnd:673,confidence_score:0.8945,wikipediaExternalRef:33173,wikidataId:"Q6368"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"executes",offsetStart:674,offsetEnd:682,confidence_score:0.503,wikipediaExternalRef:0.503,wikidataId:"Q1077724"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"malicious",offsetStart:687,offsetEnd:696,confidence_score:0.5011,wikipediaExternalRef:20901,wikidataId:"Q14001"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"web server",offsetStart:726,offsetEnd:736,confidence_score:0.8665,wikipediaExternalRef:33455,wikidataId:"Q11288"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"violates",offsetStart:767,offsetEnd:775,confidence_score:0.385,wikipediaExternalRef:1240666,wikidataId:"Q141160"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"web browser's",offsetStart:797,offsetEnd:810,confidence_score:0.6034,wikipediaExternalRef:33173,wikidataId:"Q6368"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"same-origin policy",offsetStart:811,offsetEnd:829,confidence_score:0.8835,wikipediaExternalRef:1883276,wikidataId:"Q2031810"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"scripts",offsetStart:849,offsetEnd:856,confidence_score:0.5074,wikipediaExternalRef:21490336,wikidataId:"Q187432"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"XSS",offsetStart:975,offsetEnd:978,confidence_score:0.8255,wikipediaExternalRef:241154,wikidataId:"Q371199"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"Type ",offsetStart:980,offsetEnd:986,confidence_score:0.5056,wikipediaExternalRef:41829,wikidataId:"Q7860676"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"Reflected XSS",offsetStart:988,offsetEnd:1001,confidence_score:0.5158,wikipediaExternalRef:241154,wikidataId:"Q371199"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"Persistent",offsetStart:1010,offsetEnd:1020,confidence_score:0.3757,wikipediaExternalRef:8669240,wikidataId:"Q7872822"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"HTTP request",offsetStart:1064,offsetEnd:1076,confidence_score:0.8153,wikipediaExternalRef:13443,wikidataId:"Q8777"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"HTTP response",offsetStart:1105,offsetEnd:1118,confidence_score:0.5927,wikipediaExternalRef:13443,wikidataId:"Q8777"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"Reflected XSS",offsetStart:1120,offsetEnd:1133,confidence_score:0.5158,wikipediaExternalRef:241154,wikidataId:"Q371199"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"exploits",offsetStart:1134,offsetEnd:1142,confidence_score:0.5021,wikipediaExternalRef:9875,wikidataId:"Q11287"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"attacker",offsetStart:1157,offsetEnd:1165,confidence_score:0.5111,wikipediaExternalRef:2471540,wikidataId:"Q2798820"}]->(cwe)
MERGE (cwedata)-[:HAS_ENTITY{rawName:"web application",offsetStart:,offsetEnd:,confidence_score:,wikipediaExternalRef:,wikidataId:""}]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input ',': expected "NOT" or an expression (line 47, column 68 (offset: 7134))
"MERGE (cwedata)-[:HAS_ENTITY{rawName:"web application",offsetStart:,offsetEnd:,confidence_score:,wikipediaExternalRef:,wikidataId:""}]->(cwe)"
                                                                    ^}
None


In [31]:
explore_reviews_csv_query = """
MERGE (cwe:CWE {cwe_id:"89"})
MERGE (cwedata:CWE {runtime:691})
MERGE (cwe)-[:MAPPED_WITH]->(cwedata)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.019997397708562056,source:"wikipedia-en",category:"Data-centric programming languages",page_id:925067}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.009998698833835067,source:"wikipedia-en",category:"1984 in comics",page_id:17718412}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.009998698833835067,source:"wikipedia-en",category:"Command and control",page_id:13460558}]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [32]:
explore_reviews_csv_query = """
MERGE (cwe:CWE {cwe_id:"94"})
MERGE (cwedata:CWE {runtime:833})
MERGE (cwe)-[:MAPPED_WITH]->(cwedata)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.0901926432383948,source:"wikipedia-en",category:"Computer security exploits",page_id:22272764}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.06764448218898413,source:"wikipedia-en",category:"Hacking (computer security)",page_id:11647669}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.022548160729661378,source:"wikipedia-en",category:"Programming language topics",page_id:5699671}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.022548160729661378,source:"wikipedia-en",category:"Illegal occupations",page_id:1005451}]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [33]:
explore_reviews_csv_query = """
MERGE (cwe:CWE {cwe_id:"200"})
MERGE (cwedata:CWE {runtime:1302})
MERGE (cwe)-[:MAPPED_WITH]->(cwedata)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.009935460988123061,source:"wikipedia-en",category:"Medical manuals",page_id:724491}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.009935460988123061,source:"wikipedia-en",category:"Construction",page_id:755979}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.009935460988123061,source:"wikipedia-en",category:"File sharing",page_id:802056}]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [34]:
explore_reviews_csv_query = """
MERGE (cwe:CWE {cwe_id:"284"})
MERGE (cwedata:CWE {runtime:1089})
MERGE (cwe)-[:MAPPED_WITH]->(cwedata)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.02177728820880549,source:"wikipedia-en",category:"Perimeter security",page_id:29568962}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.018774733450646813,source:"wikipedia-en",category:"Command and control",page_id:13460558}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.0054037453131323635,source:"wikipedia-en",category:"Software architecture",page_id:1805593}]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [35]:
explore_reviews_csv_query = """
MERGE (cwe:CWE {cwe_id:"294"})
MERGE (cwedata:CWE {runtime:253})
MERGE (cwe)-[:MAPPED_WITH]->(cwedata)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.09559506604831125,source:"wikipedia-en",category:"User interfaces",page_id:30875525}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.09559506604831125,source:"wikipedia-en",category:"Computing commands",page_id:28119161}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.09559506604831125,source:"wikipedia-en",category:"Computing terminology",page_id:2458835}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.047797533024155625,source:"wikipedia-en",category:"Television shows set in California",page_id:5636468}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.047797533024155625,source:"wikipedia-en",category:"2013 American television series endings",page_id:37840227}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.047797533024155625,source:"wikipedia-en",category:"2010s American reality television series",page_id:54647202}]->(cwe)


"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


# **CPE Data**

Script_CPE.cypher

Parse CPE values in CVE using cpeparser

In [36]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/CPE.json", null, {failOnError:false})
YIELD value
WITH value  AS data
//return data.CPE_Items[0].CPE.version
UNWIND data.CPE_Items as item
MERGE (cpe:CPE {version: item.CPE.version})
SET 
    cpe.part = item.CPE.part,
    cpe.vendor = item.CPE.vendor,
    cpe.product = item.CPE.product,
    cpe.update = item.CPE.update,
    cpe.edition = item.CPE.edition,
    cpe.language = item.CPE.language,
    cpe.sw_edition = item.CPE.sw_edition,
    cpe.target_sw = item.CPE.target_sw,
    cpe.target_hw = item.CPE.target_hw,
    cpe.other = item.CPE.other
    
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [37]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:host_agent:7.21:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.21"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [38]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:host_agent:7.22:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.22"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [39]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:o:microsoft:windows:-:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"-"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [40]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:702:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"702"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [41]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:750:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"750"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [42]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:752:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"752"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [43]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:753:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"753"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [44]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:754:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"754"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [45]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:755:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"755"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [46]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:756:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"756"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [47]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:731:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"731"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [48]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:740:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"740"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [49]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:751:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"751"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [50]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:757:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"757"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [51]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:702:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"702"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [52]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_krnl64nuc:7.22ext:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.22ext"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [53]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_krnl64uc:7.53:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.53"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [54]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_krnl64nuc:7.22:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.22"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [55]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_kernel:7.22:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.22"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [56]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_kernel:7.53:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.53"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [57]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_kernel:7.77:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.77"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [58]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_kernel:7.81:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.81"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [59]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_kernel:7.85:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.85"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [60]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_krnl64uc:7.22ext:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.22ext"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [61]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:700:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"700"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [62]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:701:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"701"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [63]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:710:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"710"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [64]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:711:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"711"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [65]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:730:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"730"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [66]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:731:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"731"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [67]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:740:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"740"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [68]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:750:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"750"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [69]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:751:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"751"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [70]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:752:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"752"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [71]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:753:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"753"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [72]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:754:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"754"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [73]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:755:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"755"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [74]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:756:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"756"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [75]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:757:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"757"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [76]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_kernel:7.89:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.89"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [77]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:business_objects_business_intelligence_platform:420:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"420"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [78]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:business_planning_and_consolidation:810:*:*:*:*:microsoft:*:*" })
MERGE (cpe:CPE {version:"810"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [79]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:business_planning_and_consolidation:800:*:*:*:*:microsoft:*:*" })
MERGE (cpe:CPE {version:"800"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [80]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_for_java:7.50:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.50"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [81]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:businessobjects_business_intelligence_platform:420:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"420"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [82]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:businessobjects_business_intelligence_platform:430:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"430"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [83]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:businessobjects_business_intelligence_platform:420:*:*:*:analysis:olap:*:*" })
MERGE (cpe:CPE {version:"420"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [84]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:businessobjects_business_intelligence_platform:430:*:*:*:analysis:olap:*:*" })
MERGE (cpe:CPE {version:"430"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [85]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:bank_account_management:900:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"900"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [86]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:bank_account_management:800:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"800"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [87]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:linagora:twake:*:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"*"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [88]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:o:multilaserempresas:re708_firmware:re1200r4gc-2t2r-v3_v3411b_mul029b:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"re1200r4gc-2t2r-v3_v3411b_mul029b"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [89]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:h:multilaserempresas:re708:-:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"-"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


Extract the entities from unstructured data (vulnerability description) in CVE by using OIE model, entity-fishing, and en_core_web_sm model (from spaCy)


Script_Unstructured data.cypher

In [90]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"gains",type:"verb"}),
       (node2:Frame_Argument{text:"an attacker",type:"ARG0"}),
       (node3:Reference{text:"who",type:"R-ARG0"}),
       (node4:Frame_Argument{text:"local membership",type:"ARG1"}),
       (node5:Frame_Argument{text:"to SAP_LocalAdmin",type:"ARG4"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"gains"})
MERGE (description)-[:MAPPED_WITH]->(frame)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [91]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"be",type:"verb"}),
       (node2:Frame_Argument{text:"an attacker who gains local membership to SAP_LocalAdmin",type:"ARG1"}),
       (node3:Modifier_Modal{text:"could",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"able to replace executables with a malicious file that will be started under a privileged account",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"be"})
MERGE (description)-[:MAPPED_WITH]->(frame)       
       
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [92]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"replace",type:"verb"}),
       (node2:Frame_Argument{text:"executables",type:"ARG1"}),
       (node3:Frame_Argument{text:"an attacker who gains local membership to SAP_LocalAdmin",type:"ARG2"}),
       (node4:Frame_Argument{text:"with a malicious file that will be started under a privileged account",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"replace"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [93]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"started",type:"verb"}),
       (node2:Frame_Argument{text:"a malicious file",type:"ARG1"}),
       (node3:Reference{text:"that",type:"R-ARG1"}),
       (node4:Modifier_Modal{text:"will",type:"ARGM-MOD"}),
       (node5:Modifier_Location{text:"under a privileged account",type:"ARGM-LOC"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"started"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [94]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"Note",type:"verb"}),
       (node2:Modifier_Location{text:"In SAP",type:"ARGM-LOC"}),
       (node3:Frame_Argument{text:"that by default all user members of SAP_LocaAdmin are denied the ability to logon locally by security policy so that this can only occur if the system has already been compromised",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"Note"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [95]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"denied",type:"Verb"}),
       (node2:Modifier_Manner{text:"by default all user members of",type:"ARGM-MNR"}),
       (node3:Frame_Argument{text:"the ability to logon locally by security policy",type:"ARG1"}),
       (node4:Modifier_Purpose{text:"so that this can only occur if the system has already been compromised",type:"ARGM-PRP"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"denied"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [96]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"logon",type:"Verb"}),
       (node2:Frame_Argument{text:"locally",type:"ARG1"}),
       (node3:Frame_Argument{text:"by security",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"logon"})
MERGE (description)-[:MAPPED_WITH]->(frame)       
      

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [97]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"occur",type:"Verb"}),
       (node2:Frame_Argument{text:"this",type:"ARG1"}),
       (node3:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node4:Modifier_Adverbial{text:"only",type:"ARGM-ADV"}),
       (node5:Modifier_Adverbial{text:"if the system has already been compromised",type:"ARGM-ADV"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"occur"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [98]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"compromised",type:"Verb"}),
       (node2:Frame_Argument{text:"the system",type:"ARG1"}),
       (node3:Modifier_Temporal{text:"already",type:"ARGM-TMP"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"compromised"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [99]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"encode",type:"Verb"}),
       (node2:Frame_Argument{text:"The ABAP Keyword Documentation of SAP NetWeaver Application Server - versions 702 , 731 , 740 , 750 , 751 , 752 , 753 , 754 , 755 , 756 , 757 , for ABAP and ABAP Platform",type:"ARG0"}),
       (node3:Modifier_Negation{text:"not",type:"ARGM-NEG"}),
       (node4:Modifier_Manner{text:"sufficiently",type:"ARGM-MNR"}),
       (node5:Frame_Argument{text:"user - controlled inputs",type:"ARG1"}),
       (node6:Modifier_Adverbial{text:"resulting in Cross - Site Scripting ( XSS ) vulnerability . On successful exploitation an attacker can cause limited impact on confidentiality and integrity of the application",type:"ARGM-ADV"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1),
       (node6)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0013"})
MERGE (frame:Frame {text:"encode"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [100]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"controlled",type:"Verb"}),
       (node2:Frame_Argument{text:"user",type:"ARG0"}),
       (node3:Frame_Argument{text:"inputs",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0013"})
MERGE (frame:Frame {text:"controlled"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [101]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"cause",type:"Verb"}),
       (node2:Modifier_Location{text:"On successful exploitation",type:"ARGM-LOC"}),
       (node3:Frame_Argument{text:"an attacker",type:"ARG0"}),
       (node4:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node5:Frame_Argument{text:"limited impact on confidentiality and integrity of the application",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0013"})
MERGE (frame:Frame {text:"cause"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [102]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"creates",type:"Verb"}),
       (node2:Frame_Argument{text:"SAP NetWeaver ABAP Server and ABAP Platform - versions SAP_BASIS 700 , 701 , 702 , 710 , 711 , 730 , 731 , 740 , 750 , 751 , 752 , 753 , 754 , 755 , 756 , 757 , KERNEL 7.22 , 7.53 , 7.77 , 7.81 , 7.85 , 7.89 , KRNL64UC 7.22 , 7.22EXT , 7.53 , KRNL64NUC 7.22 , 7.22EXT",type:"ARG0"}),
       (node3:Frame_Argument{text:"information about system identity",type:"ARG1"}),
       (node4:Modifier_Location{text:"in an ambiguous format",type:"ARGM-LOC"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0014"})
MERGE (frame:Frame {text:"creates"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [103]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"lead",type:"Verb"}),
       (node2:Frame_Argument{text:"This",type:"ARG0"}),
       (node3:Modifier_Modal{text:"could",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"to capture - replay vulnerability",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0014"})
MERGE (frame:Frame {text:"lead"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [104]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"may",type:"Verb"}),
       (node2:Modifier_Modal{text:"could",type:"ARGM-MOD"}),
       (node3:Frame_Argument{text:"be exploited by malicious users to obtain illegitimate access to the system",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0014"})
MERGE (frame:Frame {text:"may"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [105]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"exploited",type:"Verb"}),
       (node2:Frame_Argument{text:"This",type:"ARG1"}),
       (node3:Modifier_Modal{text:"may",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"by malicious users",type:"ARG0"}),
       (node5:Frame_Argument{text:"to obtain illegitimate access to the system",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0014"})
MERGE (frame:Frame {text:"exploited"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [106]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"return",type:"Verb"}),
       (node2:Frame_Argument{text:"some calls",type:"ARG0"}),
       (node3:Frame_Argument{text:"json",type:"ARG1"}),
       (node4:Modifier_Manner{text:"with wrong content type",type:"ARGM-MNR"}),
       (node5:Modifier_Manner{text:"in the header of the",type:"ARGM-MNR"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0015"})
MERGE (frame:Frame {text:"return"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [107]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"calls",type:"Verb"}),
       (node2:Frame_Argument{text:"a custom application",type:"ARG0"}),
       (node3:Reference{text:"that",type:"R-ARG0"}),
       (node4:Modifier_Manner{text:"directly",type:"ARGM-MNR"}),
       (node5:Frame_Argument{text:"the jsp of Web Intelligence",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0015"})
MERGE (frame:Frame {text:"calls"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [108]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"be",type:"Verb"}),
       (node2:Frame_Argument{text:"a custom application that calls directly the jsp of Web Intelligence DHTML",type:"ARG1"}),
       (node3:Modifier_Modal{text:"may",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"vulnerable to XSS attacks",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0015"})
MERGE (frame:Frame {text:"be"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [109]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"cause",type:"Verb"}),
       (node2:Modifier_Location{text:"On successful exploitation",type:"ARGM-LOC"}),
       (node3:Frame_Argument{text:"an attacker",type:"ARG0"}),
       (node4:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node5:Frame_Argument{text:"limited impact on confidentiality and integrity of the application",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0015"})
MERGE (frame:Frame {text:"cause"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [110]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"allows",type:"Verb"}),
       (node2:Frame_Argument{text:"SAP BPC MS 10.0 - version 810",type:"ARG0"}),
       (node3:Frame_Argument{text:"an unauthorized attacker to execute crafted database queries",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0016"})
MERGE (frame:Frame {text:"allows"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [111]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"execute",type:"Verb"}),
       (node2:Frame_Argument{text:"an unauthorized attacker",type:"ARG0"}),
       (node3:Frame_Argument{text:"crafted database queries",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0016"})
MERGE (frame:Frame {text:"execute"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [112]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"lead",type:"Verb"}),
       (node2:Frame_Argument{text:"The exploitation of this issue",type:"ARG0"}),
       (node3:Modifier_Modal{text:"could",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"to SQL injection vulnerability",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0016"})
MERGE (frame:Frame {text:"lead"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [113]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"could",type:"Verb"}),
       (node2:Frame_Argument{text:"The exploitation of this issue",type:"ARG0"}),
       (node3:Modifier_Modal{text:"could",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"allow an attacker to access , modify , and/or delete data from the backend database",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0016"})
MERGE (frame:Frame {text:"could"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [114]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"allow",type:"Verb"}),
       (node2:Frame_Argument{text:"The exploitation of this issue",type:"ARG0"}),
       (node3:Modifier_Modal{text:"could",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"an attacker to access , modify , and/or delete data from the backend database",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0016"})
MERGE (frame:Frame {text:"allow"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [115]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"can",type:"Verb"}),
       (node2:Frame_Argument{text:"An unauthenticated attacker in SAP NetWeaver AS for Java - version 7.50",type:"ARG0"}),
       (node3:Frame_Argument{text:"attach to an open interface and make use of an open naming and directory API to access services which can be used to perform unauthorized operations affecting users and data on the current",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"can"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [116]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"attach",type:"Verb"}),
       (node2:Frame_Argument{text:"An unauthenticated attacker in SAP NetWeaver AS for Java - version 7.50 , due to improper access control",type:"ARG0"}),
       (node3:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"to an open interface",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"attach"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [117]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"make",type:"Verb"}),
       (node2:Frame_Argument{text:"An unauthenticated attacker in SAP NetWeaver AS for Java - version 7.50 , due to improper access control",type:"ARG0"}),
       (node3:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"use of an open naming and directory API to access services which can be used to perform unauthorized operations affecting users and data on the current system",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"make"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [118]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"used",type:"Verb"}),
       (node2:Frame_Argument{text:"access services",type:"ARG1"}),
       (node3:Reference{text:"which",type:"R-ARG1"}),
       (node4:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node5:Frame_Argument{text:"to perform unauthorized operations affecting users and data on the current system",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"used"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [119]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"perform",type:"Verb"}),
       (node2:Frame_Argument{text:"access services",type:"ARG0"}),
       (node3:Frame_Argument{text:"unauthorized operations affecting users and data on the current system",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"perform"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [120]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"affecting",type:"Verb"}),
       (node2:Frame_Argument{text:"unauthorized operations",type:"ARG0"}),
       (node3:Frame_Argument{text:"users",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"affecting"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [121]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"could",type:"Verb"}),
       (node2:Frame_Argument{text:"This",type:"ARG0"}),
       (node3:Frame_Argument{text:"allow the attacker to have full read access to user data , make modifications to user data , and make services within the system unavailable",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"could"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [122]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"allow",type:"Verb"}),
       (node2:Frame_Argument{text:"This",type:"ARG0"}),
       (node3:Modifier_Modal{text:"could",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"the attacker to have full read access to user data , make modifications to user data , and make services within the system unavailable",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"allow"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [123]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"have",type:"Verb"}),
       (node2:Frame_Argument{text:"the attacker",type:"ARG0"}),
       (node3:Frame_Argument{text:"full read access to user data",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"have"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [124]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"user",type:"Verb"}),
       (node2:Frame_Argument{text:"full read",type:"ARG0"}),
       (node3:Frame_Argument{text:"data",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"user"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [125]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"make",type:"Verb"}),
       (node2:Frame_Argument{text:"the attacker",type:"ARG0"}),
       (node3:Frame_Argument{text:"modifications to user data , and make",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"make"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [126]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"make",type:"Verb"}),
       (node2:Frame_Argument{text:"the attacker",type:"ARG0"}),
       (node3:Frame_Argument{text:"services within the system",type:"ARG1"}),
       (node4:Modifier_Manner{text:"unavailable",type:"ARGM-MNR"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"make"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [127]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"controlled",type:"Verb"}),
       (node2:Frame_Argument{text:"user",type:"ARG0"}),
       (node3:Frame_Argument{text:"input",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"controlled"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [128]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"modify",type:"Verb"}),
       (node2:Modifier_Cause{text:"Due to improper input sanitization of user - controlled input in SAP BusinessObjects Business Intelligence",type:"ARGM-CAU"}),
       (node3:Frame_Argument{text:"an attacker with basic user - level privileges",type:"ARG0"}),
       (node4:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node5:Frame_Argument{text:"reports containing a malicious payload",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"modify"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [129]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"containing",type:"Verb"}),
       (node2:Frame_Argument{text:"reports",type:"ARG0"}),
       (node3:Frame_Argument{text:"a malicious payload",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"containing"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [130]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"are",type:"Verb"}),
       (node2:Frame_Argument{text:"these reports",type:"ARG1"}),
       (node3:Frame_Argument{text:"viewable",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"are"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [131]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"opens",type:"Verb"}),
       (node2:Frame_Argument{text:"anyone",type:"ARG0"}),
       (node3:Reference{text:"who",type:"R-ARG0"}),
       (node4:Frame_Argument{text:"those reports",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"opens"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [132]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"be",type:"Verb"}),
       (node2:Frame_Argument{text:"anyone who opens those reports",type:"ARG1"}),
       (node3:Modifier_Modal{text:"would",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"susceptible to stored XSS attacks",type:"ARG2"}),
       (node5:Modifier_Cause{text:"As",type:"ARGM-CAU"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"be"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [133]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"maintained",type:"Verb"}),
       (node2:Frame_Argument{text:"information",type:"ARG1"}),
       (node3:Modifier_Location{text:"in the victim 's web browser",type:"ARGM-LOC"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"maintained"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [134]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"read",type:"Verb"}),
       (node2:Modifier_Cause{text:"As a result of the attack",type:"ARGM-CAU"}),
       (node3:Frame_Argument{text:"information maintained in the victim 's web browser",type:"ARG1"}),
       (node4:Modifier_Modal{text:"can",type:"ARGM-MOD"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"read"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [135]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"modified",type:"Verb"}),
       (node2:Frame_Argument{text:"information maintained in the victim 's web browser",type:"ARG1"}),
       (node3:Modifier_Modal{text:"can",type:"ARGM-MOD"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"modified"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [136]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"sent",type:"Verb"}),
       (node2:Frame_Argument{text:"information maintained in the victim 's web browser",type:"ARG1"}),
       (node3:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"to the attacker",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"sent"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [137]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"allows",type:"Verb"}),
       (node2:Frame_Argument{text:"SAP BusinessObjects Business Intelligence Analysis edition for OLAP",type:"ARG0"}),
       (node3:Frame_Argument{text:"an authenticated attacker to inject malicious code that can be executed by the application over the network . On successful exploitation , an attacker can perform operations that may completely compromise the application causing a high impact on the confidentiality , integrity , and availability of the application",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0022"})
MERGE (frame:Frame {text:"allows"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [138]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"inject",type:"Verb"}),
       (node2:Frame_Argument{text:"an authenticated attacker",type:"ARG0"}),
       (node3:Frame_Argument{text:"malicious code that can be executed by the application over the network . On successful exploitation",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0022"})
MERGE (frame:Frame {text:"inject"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [139]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"executed",type:"Verb"}),
       (node2:Frame_Argument{text:"malicious code",type:"ARG1"}),
       (node3:Reference{text:"that",type:"R-ARG1"}),
       (node4:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node5:Frame_Argument{text:"by the application over the network",type:"ARG0"}),
      
       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0022"})
MERGE (frame:Frame {text:"executed"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [140]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"perform",type:"Verb"}),
       (node2:Frame_Argument{text:"an attacker",type:"ARG0"}),
       (node3:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"operations",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0022"})
MERGE (frame:Frame {text:"perform"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [141]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"compromise",type:"Verb"}),
       (node2:Frame_Argument{text:"operations",type:"ARG0"}),
       (node3:Reference{text:"that",type:"R-ARG0"}),
       (node4:Modifier_Modal{text:"may",type:"ARGM-MOD"}),
       (node5:Modifier_Extent{text:"completely",type:"ARGM-EXT"}),
       (node6:Frame_Argument{text:"the application causing a high impact on the confidentiality , integrity , and availability of the application",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1),
       (node6)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0022"})
MERGE (frame:Frame {text:"compromise"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [142]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"causing",type:"Verb"}),
       (node2:Frame_Argument{text:"the application",type:"ARG0"}),
       (node3:Frame_Argument{text:"a high impact on the confidentiality , integrity , and availability of the application",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0022"})
MERGE (frame:Frame {text:"causing"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [143]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"clicks",type:"Verb"}),
       (node2:Modifier_Temporal{text:"when",type:"ARGM-TMP"}),
       (node3:Frame_Argument{text:"a user",type:"ARG0"}),
       (node4:Frame_Argument{text:"a smart link to navigate to another app",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0023"})
MERGE (frame:Frame {text:"clicks"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [144]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"navigate",type:"Verb"}),
       (node2:Frame_Argument{text:"a user",type:"ARG0"}),
       (node3:Frame_Argument{text:"to another app",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0023"})
MERGE (frame:Frame {text:"navigate"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [145]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"shown",type:"Verb"}),
       (node2:Modifier_Location{text:"In SAP Bank Account Management ( Manage Banks ) application",type:"ARGM-LOC"}),
       (node3:Modifier_Temporal{text:"when a user clicks a smart link to navigate to another app",type:"ARGM-TMP"}),
       (node4:Frame_Argument{text:"personal data",type:"ARG1"}),
       (node5:Modifier_Manner{text:"directly",type:"ARGM-MNR"}),
       (node6:Modifier_Location{text:"in the URL",type:"ARGM-LOC"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1), 
       (node5)-[:HAS_ARGUMENT]->(node1),
       (node6)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0023"})
MERGE (frame:Frame {text:"shown"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [146]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"get",type:"Verb"}),
       (node2:Frame_Argument{text:"They",type:"ARG1"}),
       (node3:Modifier_Modal{text:"might",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"captured in log files , bookmarks , and so on disclosing sensitive data of the application",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0023"})
MERGE (frame:Frame {text:"get"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [147]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"captured",type:"Verb"}),
       (node2:Frame_Argument{text:"They",type:"ARG1"}),
       (node3:Modifier_Modal{text:"might",type:"ARGM-MOD"}),
       (node4:Modifier_Location{text:"in log files , bookmarks",type:"ARGM-LOC"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0023"})
MERGE (frame:Frame {text:"captured"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [148]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"Stored",type:"Verb"}),
       (node2:Frame_Argument{text:")",type:"ARG0"}),
       (node3:Frame_Argument{text:"in GitHub repository linagora / twake",type:"ARG1"}),
       (node4:Modifier_Temporal{text:"prior to 2023.Q1.1200 +",type:"ARGM-TMP"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0028"})
MERGE (frame:Frame {text:"Stored"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [149]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"found",type:"Verb"}),
       (node2:Frame_Argument{text:"A vulnerability",type:"ARG1"}),
       (node3:Modifier_Location{text:"in Multilaser RE708 RE1200R4GC-2T2R - V3_v3411b_MUL029B.",type:"ARGM-LOC"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0029"})
MERGE (frame:Frame {text:"found"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [150]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"rated",type:"Verb"}),
       (node2:Frame_Argument{text:"in Multilaser RE708 RE1200R4GC-2T2R - V3_v3411b_MUL029B",type:"ARG2"}),
       (node3:Frame_Argument{text:"It",type:"ARG1"}),
       (node4:Frame_Argument{text:"as problematic",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0029"})
MERGE (frame:Frame {text:"rated"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [151]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"affects",type:"Verb"}),
       (node2:Frame_Argument{text:"This issue",type:"ARG0"}),
       (node3:Frame_Argument{text:"some unknown processing of the component Telnet Service",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0029"})
MERGE (frame:Frame {text:"affects"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [152]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"leads",type:"Verb"}),
       (node2:Frame_Argument{text:"The manipulation",type:"ARG0"}),
       (node3:Frame_Argument{text:"to denial of service",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0029"})
MERGE (frame:Frame {text:"leads"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [153]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"initiated",type:"Verb"}),
       (node2:Frame_Argument{text:"The attack",type:"ARG1"}),
       (node3:Modifier_Modal{text:"may",type:"ARGM-MOD"}),
       (node4:Modifier_Manner{text:"remotely",type:"ARGM-MNR"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0029"})
MERGE (frame:Frame {text:"initiated"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [154]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"assigned",type:"Verb"}),
       (node2:Frame_Argument{text:"The identifier VDB-217169",type:"ARG1"}),
       (node3:Frame_Argument{text:"to this vulnerability",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0029"})
MERGE (frame:Frame {text:"assigned"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]
